# 🔬 Claude Tools - Interactive Debug Processor

This notebook provides cell-by-cell execution for debugging and analysis.

Each cell can be run independently to:
- Debug specific functions
- Analyze request/response data
- Monitor performance
- Test code snippets

## 📦 1. Setup and Dependencies

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Drive mounted")

In [ ]:
# Install dependencies
!pip install -q google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
print("✅ Dependencies installed")

# Import libraries
import os
import json
import time
import traceback
from datetime import datetime
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
import io
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML, clear_output

## 🔧 2. Initialize Processor

In [ ]:
# Configuration
FOLDER_ID = "1tzHn4J3QntSLJlJNXcNJe3cLdILGEb3Z"

# Initialize authentication
auth.authenticate_user()
drive_service = build('drive', 'v3')
print(f"✅ Authenticated and ready")
print(f"📁 Monitoring folder: {FOLDER_ID}")

In [ ]:
# Processor class with debug features
class DebugProcessor:
    def __init__(self, folder_id, drive_service):
        self.folder_id = folder_id
        self.drive_service = drive_service
        self.processed_requests = set()
        self.execution_history = []
        self.debug_mode = True
        
    def list_all_files(self):
        """List all files in the folder for debugging"""
        query = f"'{self.folder_id}' in parents and trashed=false"
        results = self.drive_service.files().list(
            q=query,
            fields="files(id, name, createdTime, size)",
            orderBy="createdTime desc"
        ).execute()
        
        files = results.get('files', [])
        if self.debug_mode:
            print(f"📁 Found {len(files)} files in Drive folder")
            
        return files
    
    def get_pending_requests(self):
        """Get unprocessed requests"""
        all_files = self.list_all_files()
        requests = []
        
        for file in all_files:
            if (file['name'].startswith('command_') and 
                file['name'].endswith('.json') and
                file['id'] not in self.processed_requests):
                requests.append(file)
                
        if self.debug_mode:
            print(f"📨 Found {len(requests)} pending requests")
            
        return requests
    
    def read_request(self, file_id, file_name=None):
        """Read and parse request with debugging"""
        try:
            content = self.drive_service.files().get_media(fileId=file_id).execute()
            data = json.loads(content.decode('utf-8'))
            
            if self.debug_mode:
                print(f"\n📖 Read request: {file_name or file_id}")
                print(f"   Type: {data.get('type')}")
                print(f"   Timestamp: {data.get('timestamp')}")
                if 'code' in data:
                    print(f"   Code length: {len(data['code'])} chars")
                    
            return data
        except Exception as e:
            print(f"❌ Error reading request: {e}")
            return None
    
    def execute_code_safe(self, code):
        """Execute code with comprehensive error handling"""
        from io import StringIO
        import sys
        
        # Store original stdout
        old_stdout = sys.stdout
        sys.stdout = StringIO()
        
        start_time = time.time()
        
        try:
            # Create isolated namespace
            namespace = {'__name__': '__main__'}
            exec(code, namespace)
            
            output = sys.stdout.getvalue()
            execution_time = time.time() - start_time
            
            result = {
                'status': 'success',
                'output': output,
                'execution_time': execution_time,
                'timestamp': time.time()
            }
            
            if self.debug_mode:
                print(f"✅ Execution successful ({execution_time:.2f}s)")
                
        except Exception as e:
            execution_time = time.time() - start_time
            
            result = {
                'status': 'error',
                'error': str(e),
                'error_type': type(e).__name__,
                'traceback': traceback.format_exc(),
                'execution_time': execution_time,
                'timestamp': time.time()
            }
            
            if self.debug_mode:
                print(f"❌ Execution failed: {type(e).__name__}: {e}")
                
        finally:
            sys.stdout = old_stdout
            
        # Store in history
        self.execution_history.append(result)
        
        return result
    
    def write_response(self, command_id, response_data):
        """Write response with debugging"""
        response_name = f'result_{command_id}.json'
        
        try:
            file_metadata = {
                'name': response_name,
                'parents': [self.folder_id]
            }
            
            media = MediaIoBaseUpload(
                io.BytesIO(json.dumps(response_data, indent=2).encode('utf-8')),
                mimetype='application/json'
            )
            
            file = self.drive_service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id,name'
            ).execute()
            
            if self.debug_mode:
                print(f"✅ Response written: {response_name}")
                print(f"   File ID: {file['id']}")
                
            return file
            
        except Exception as e:
            print(f"❌ Error writing response: {e}")
            return None

# Create processor instance
processor = DebugProcessor(FOLDER_ID, drive_service)
print("✅ Debug processor initialized")

## 🔍 3. Debug Functions

In [ ]:
# List all files in the Drive folder
files = processor.list_all_files()

# Create DataFrame for better visualization
if files:
    df = pd.DataFrame(files)
    df['createdTime'] = pd.to_datetime(df['createdTime'])
    df['size'] = df['size'].astype(int) if 'size' in df else 0
    
    print("\n📊 Files in Drive folder:")
    display(df[['name', 'createdTime', 'size']].head(10))
else:
    print("No files found")

In [ ]:
# Check pending requests
pending = processor.get_pending_requests()

if pending:
    print("\n📋 Pending requests:")
    for i, req in enumerate(pending[:5]):
        print(f"{i+1}. {req['name']} (created: {req['createdTime']})")
else:
    print("No pending requests")

## 🧪 4. Test Code Execution

In [ ]:
# Test the execution engine with sample code
test_code = '''
import numpy as np
import sys

print("🧪 Test execution from debug notebook")
print(f"Python version: {sys.version}")

# Test computation
arr = np.random.rand(100, 100)
result = np.mean(arr)
print(f"Mean of random array: {result:.4f}")

# Test error handling
try:
    import torch
    if torch.cuda.is_available():
        print(f"GPU available: {torch.cuda.get_device_name(0)}")
    else:
        print("No GPU available")
except ImportError:
    print("PyTorch not installed")
'''

print("⚡ Executing test code...")
result = processor.execute_code_safe(test_code)

print("\n📋 Execution Result:")
print(f"Status: {result['status']}")
print(f"Execution time: {result['execution_time']:.3f}s")
print("\nOutput:")
print("-" * 40)
print(result.get('output', result.get('error', 'No output')))
print("-" * 40)

## 🔄 5. Process Single Request (Step by Step)

In [ ]:
# Process one request with full debugging
pending = processor.get_pending_requests()

if pending:
    # Take first request
    request_file = pending[0]
    print(f"\n🎯 Processing: {request_file['name']}")
    
    # Step 1: Read request
    request_data = processor.read_request(request_file['id'], request_file['name'])
    
    if request_data:
        # Step 2: Show request details
        print("\n📝 Request details:")
        print(json.dumps({k: v for k, v in request_data.items() if k != 'code'}, indent=2))
        
        if 'code' in request_data:
            print("\n📄 Code to execute:")
            print("-" * 40)
            print(request_data['code'][:200] + "..." if len(request_data['code']) > 200 else request_data['code'])
            print("-" * 40)
            
            # Step 3: Execute
            print("\n⚡ Executing...")
            result = processor.execute_code_safe(request_data['code'])
            
            # Step 4: Write response
            command_id = request_file['name'].replace('command_', '').replace('.json', '')
            response_file = processor.write_response(command_id, result)
            
            # Step 5: Mark as processed
            processor.processed_requests.add(request_file['id'])
            
            print("\n✅ Request processed successfully!")
else:
    print("No pending requests to process")

## 📊 6. Performance Analysis

In [ ]:
# Analyze execution history
if processor.execution_history:
    # Create DataFrame from history
    history_df = pd.DataFrame(processor.execution_history)
    
    # Summary statistics
    print("📊 Execution Statistics:")
    print(f"Total executions: {len(history_df)}")
    print(f"Success rate: {(history_df['status'] == 'success').mean():.1%}")
    print(f"Average execution time: {history_df['execution_time'].mean():.3f}s")
    
    # Plot execution times
    if len(history_df) > 1:
        plt.figure(figsize=(10, 4))
        plt.plot(history_df.index, history_df['execution_time'], 'b-o')
        plt.xlabel('Execution Number')
        plt.ylabel('Time (seconds)')
        plt.title('Execution Time History')
        plt.grid(True, alpha=0.3)
        plt.show()
else:
    print("No execution history yet")

## 🔁 7. Continuous Processing Loop (with Live Updates)

In [ ]:
# Interactive processing loop with live updates
def process_with_live_updates(duration=60, poll_interval=5):
    """Process requests with live dashboard"""
    start_time = time.time()
    stats = {'processed': 0, 'errors': 0, 'total_time': 0}
    
    print(f"🚀 Starting live processor for {duration}s...")
    print("Press 'Interrupt' to stop\n")
    
    try:
        while time.time() - start_time < duration:
            # Clear output for live update
            clear_output(wait=True)
            
            # Display dashboard
            elapsed = int(time.time() - start_time)
            remaining = duration - elapsed
            
            print("🎯 Claude Tools Live Processor")
            print("=" * 40)
            print(f"⏱️  Elapsed: {elapsed}s | Remaining: {remaining}s")
            print(f"📊 Processed: {stats['processed']} | Errors: {stats['errors']}")
            print(f"⚡ Avg time: {stats['total_time']/(stats['processed'] or 1):.3f}s")
            print("=" * 40)
            
            # Get pending requests
            pending = processor.get_pending_requests()
            
            if pending:
                print(f"\n📨 Processing {len(pending)} requests...")
                
                for req in pending:
                    # Process request
                    request_data = processor.read_request(req['id'])
                    
                    if request_data and 'code' in request_data:
                        result = processor.execute_code_safe(request_data['code'])
                        
                        # Write response
                        command_id = req['name'].replace('command_', '').replace('.json', '')
                        processor.write_response(command_id, result)
                        
                        # Update stats
                        stats['processed'] += 1
                        stats['total_time'] += result.get('execution_time', 0)
                        
                        if result['status'] == 'error':
                            stats['errors'] += 1
                        
                        # Mark as processed
                        processor.processed_requests.add(req['id'])
            else:
                print("\n⏳ No pending requests...")
            
            # Wait before next poll
            time.sleep(poll_interval)
            
    except KeyboardInterrupt:
        print("\n\n🛑 Stopped by user")
    
    # Final summary
    print("\n" + "=" * 40)
    print("📊 Session Summary:")
    print(f"Total processed: {stats['processed']}")
    print(f"Success rate: {(stats['processed']-stats['errors'])/max(stats['processed'],1):.1%}")
    print(f"Total runtime: {int(time.time()-start_time)}s")

# Run for 5 minutes
process_with_live_updates(duration=300, poll_interval=3)

## 🛠️ 8. Debug Tools

In [ ]:
# Inspect a specific request file
def inspect_request(file_name):
    """Detailed inspection of a request file"""
    files = processor.list_all_files()
    
    for file in files:
        if file['name'] == file_name:
            print(f"📄 Inspecting: {file_name}")
            print(f"ID: {file['id']}")
            print(f"Created: {file['createdTime']}")
            
            # Read content
            content = processor.drive_service.files().get_media(fileId=file['id']).execute()
            data = json.loads(content.decode('utf-8'))
            
            print("\nContent:")
            print(json.dumps(data, indent=2))
            return data
    
    print(f"File '{file_name}' not found")
    return None

# Example: inspect_request('command_claude_1234567890_1234567890.json')

In [ ]:
# Clean up old files
def cleanup_old_files(older_than_hours=24):
    """Remove old request/response files"""
    files = processor.list_all_files()
    cutoff_time = datetime.now() - pd.Timedelta(hours=older_than_hours)
    
    to_delete = []
    for file in files:
        if file['name'].startswith(('command_', 'result_')):
            created = pd.to_datetime(file['createdTime']).tz_localize(None)
            if created < cutoff_time:
                to_delete.append(file)
    
    if to_delete:
        print(f"🗑️  Found {len(to_delete)} old files to clean up")
        
        # Uncomment to actually delete:
        # for file in to_delete:
        #     processor.drive_service.files().delete(fileId=file['id']).execute()
        #     print(f"Deleted: {file['name']}")
    else:
        print("✅ No old files to clean up")

# Check what would be cleaned up
cleanup_old_files(older_than_hours=1)

## 💡 9. Advanced Features

In [ ]:
# Create a test request manually
def create_test_request(code):
    """Create a test request in Drive"""
    request_data = {
        'type': 'execute',
        'code': code,
        'timestamp': time.time(),
        'source': 'debug_notebook'
    }
    
    command_id = f"test_{int(time.time())}"
    file_name = f"command_{command_id}.json"
    
    file_metadata = {
        'name': file_name,
        'parents': [FOLDER_ID]
    }
    
    media = MediaIoBaseUpload(
        io.BytesIO(json.dumps(request_data, indent=2).encode('utf-8')),
        mimetype='application/json'
    )
    
    file = drive_service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id,name'
    ).execute()
    
    print(f"✅ Created test request: {file_name}")
    print(f"   File ID: {file['id']}")
    return file

# Example test
test_file = create_test_request('''
print("🧪 Test from debug notebook!")
print(f"Current time: {datetime.now()}")
''')

In [ ]:
# Monitor execution performance over time
def plot_performance_metrics():
    """Visualize processor performance"""
    if not processor.execution_history:
        print("No execution history to plot")
        return
    
    df = pd.DataFrame(processor.execution_history)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # Execution time plot
    axes[0].plot(df['timestamp'], df['execution_time'], 'b-o', label='Execution Time')
    axes[0].set_ylabel('Time (seconds)')
    axes[0].set_title('Execution Time per Request')
    axes[0].grid(True, alpha=0.3)
    axes[0].legend()
    
    # Success/Error rate
    success_rate = df.groupby(df['timestamp'].dt.floor('T'))['status'].apply(
        lambda x: (x == 'success').mean() * 100
    )
    
    axes[1].plot(success_rate.index, success_rate.values, 'g-o', label='Success Rate')
    axes[1].set_ylabel('Success Rate (%)')
    axes[1].set_xlabel('Time')
    axes[1].set_title('Success Rate Over Time')
    axes[1].grid(True, alpha=0.3)
    axes[1].legend()
    axes[1].set_ylim(0, 105)
    
    plt.tight_layout()
    plt.show()

# Plot performance
plot_performance_metrics()

## 🎯 Summary

This interactive notebook provides:
- **Cell-by-cell debugging** - Run each part independently
- **Visual monitoring** - See requests and responses in real-time
- **Performance analysis** - Track execution times and success rates
- **Debug tools** - Inspect, test, and clean up files
- **Live dashboard** - Watch processing happen in real-time

Use this notebook for:
1. **Development** - Test new features
2. **Debugging** - Trace issues step by step
3. **Monitoring** - Watch system performance
4. **Analysis** - Understand usage patterns